In [1]:
! pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 36.1 MB/s 
     |████████████████████████████████| 163 kB 69.7 MB/s 
     |████████████████████████████████| 7.6 MB 48.0 MB/s 


In [2]:
from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification
import pandas as pd

In [3]:
CATEGORIES = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family",
              "fantasy", "film_noir", "history", "horror", "music", "musical", "mystery", "romance", "sci_fi",
              "short", "sport", "superhero", "thriller", "war", "western"]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_name_or_path = "facebook/convnext-tiny-224"

In [7]:
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

In [39]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io, color
from torch.utils.data import (
    Dataset,
    DataLoader
)

class FilmFeatureExtractor(object):
    def __call__(self, image, targets):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = targets
        
        return sample

class FilmDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(os.path.join(self.root_dir, csv_file))
        self.shape = self.annotations.shape
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index]['poster_path'])
        image = io.imread(img_path)
        if len(image.shape) < 3:
          image = color.gray2rgb(image)
        labels = torch.tensor(self.annotations.iloc[index][CATEGORIES])
        data = self.transform(image, labels)
        
        return data

df = FilmDataset(
    csv_file="data/preprocessed.csv",
    root_dir="drive/MyDrive",
    transform=FilmFeatureExtractor(),
)


In [40]:
df.__getitem__(2137)

{'pixel_values': tensor([[[[ 0.0398,  0.1083,  0.1768,  ..., -1.9124, -1.9638, -1.9467],
          [-0.0458,  0.0398,  0.1083,  ..., -1.8097, -1.8268, -1.7925],
          [-0.1143, -0.0287,  0.0227,  ..., -1.8953, -1.8097, -1.6042],
          ...,
          [-1.4843, -1.3987, -1.2959,  ..., -1.8097, -1.3987, -1.1760],
          [-1.4843, -1.4500, -1.3815,  ..., -1.7754, -1.5870, -1.6213],
          [-1.5699, -1.5528, -1.5185,  ..., -1.7925, -1.7583, -1.8782]],

         [[ 0.1702,  0.2402,  0.3102,  ..., -1.8256, -1.8782, -1.8606],
          [ 0.0826,  0.1702,  0.2402,  ..., -1.7206, -1.7381, -1.7031],
          [ 0.0126,  0.1001,  0.1527,  ..., -1.8081, -1.7206, -1.5105],
          ...,
          [-1.3880, -1.3004, -1.1954,  ..., -1.7206, -1.3004, -1.0728],
          [-1.3880, -1.3529, -1.2829,  ..., -1.6856, -1.4930, -1.5280],
          [-1.4755, -1.4580, -1.4230,  ..., -1.7031, -1.6681, -1.7906]],

         [[ 0.3916,  0.4614,  0.5311,  ..., -1.5953, -1.6476, -1.6302],
          [ 0

In [44]:
im = io.imread("drive/MyDrive/data/posters/short/need.jpg")
color.gray2rgb(im)

array([[[42, 42, 42],
        [25, 25, 25],
        [52, 52, 52],
        ...,
        [ 7,  7,  7],
        [ 7,  7,  7],
        [ 7,  7,  7]],

       [[25, 25, 25],
        [21, 21, 21],
        [48, 48, 48],
        ...,
        [ 7,  7,  7],
        [ 7,  7,  7],
        [ 7,  7,  7]],

       [[16, 16, 16],
        [ 5,  5,  5],
        [35, 35, 35],
        ...,
        [ 7,  7,  7],
        [ 7,  7,  7],
        [ 7,  7,  7]],

       ...,

       [[47, 47, 47],
        [50, 50, 50],
        [60, 60, 60],
        ...,
        [28, 28, 28],
        [61, 61, 61],
        [39, 39, 39]],

       [[22, 22, 22],
        [33, 33, 33],
        [51, 51, 51],
        ...,
        [42, 42, 42],
        [66, 66, 66],
        [30, 30, 30]],

       [[15, 15, 15],
        [20, 20, 20],
        [26, 26, 26],
        ...,
        [39, 39, 39],
        [29, 29, 29],
        [32, 32, 32]]], dtype=uint8)

## Train test split

In [48]:
train_size = int(0.4 * df.shape[0])
eval_size = int(0.4 * df.shape[0])
test_size = df.shape[0] - train_size - eval_size

print(df.shape[0], train_size, eval_size, test_size)

4165 1666 1666 833


In [49]:
df_train, df_eval, df_test = torch.utils.data.random_split(df, [train_size, eval_size, test_size])

## Model

In [70]:
model = ConvNextForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(CATEGORIES),
    ignore_mismatched_sizes=True
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--convnext-tiny-224/snapshots/0d1c8dedaa107d4ae537c5b10e5cd0a8c865e84e/config.json
Model config ConvNextConfig {
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    9,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    96,
    192,
    384,
    768
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23"
  },
  "image_size": 224,
  "initializer_range": 

In [18]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="drive/MyDrive/data/img", 
    evaluation_strategy="steps"
)

In [19]:
! pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 31.2 MB/s 
     |████████████████████████████████| 212 kB 77.3 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 115 kB 72.1 MB/s 
     |████████████████████████████████| 127 kB 71.2 MB/s 
     |████████████████████████████████| 115 kB 70.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [21]:
def compute_metrics(eval_pred):
    print(eval_pred)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [22]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.stack([x['labels'][0] for x in batch])
    }

In [51]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_eval,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [52]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1666
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 627
  Number of trainable parameters = 27838584


Step,Training Loss,Validation Loss,Accuracy
500,0.621000,0.460735,0.810324


***** Running Evaluation *****
  Num examples = 1666
  Batch size = 8


Saving model checkpoint to drive/MyDrive/data/img/checkpoint-500
Configuration saved in drive/MyDrive/data/img/checkpoint-500/config.json
Model weights saved in drive/MyDrive/data/img/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=627, training_loss=0.5671912684585109, metrics={'train_runtime': 284.6719, 'train_samples_per_second': 17.557, 'train_steps_per_second': 2.203, 'total_flos': 1.2566430533409178e+17, 'train_loss': 0.5671912684585109, 'epoch': 3.0})

In [73]:
model = ConvNextForImageClassification.from_pretrained(os.path.join(training_args.output_dir, 'checkpoint-500'))

loading configuration file drive/MyDrive/data/img/checkpoint-500/config.json
Model config ConvNextConfig {
  "_name_or_path": "facebook/convnext-tiny-224",
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    9,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    96,
    192,
    384,
    768
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 

In [87]:
poster = df_test.__getitem__(100)
with torch.no_grad():
  logits = model(pixel_values = poster['pixel_values']).logits
print(poster['labels'])
print(logits)

tensor([1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[ 4.9351,  4.2542, -2.3278, -1.6684, -1.7381, -2.4925, -1.9073, -1.7978,
         -1.6811, -1.9966, -1.5549, -2.1642, -1.9821, -1.9012, -1.9062, -1.9209,
         -1.8538, -2.0351, -2.0130, -1.7774, -2.1877, -1.5690, -1.8366, -1.9125]])
